In [ ]:
import pandas as pd
import numpy as np
import json
import io
import datetime as dt
import string
import unicodedata

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

import re
from bs4 import BeautifulSoup


from gensim import corpora, models, similarities

# Read in data

In [ ]:
import random
import json
text_dictionary = {}
i = 0
index = 0
with open('C:/Users/utkar/Downloads/large_data.json') as f:
    line = f.readline()
    while line:
        index += 1
        print("I = " + str(index))
        loaded_json = json.loads(line)
        if 'uigoogle' in loaded_json:
            reviews = loaded_json['uigoogle']['reviews']
            for review in reviews:
                text_dictionary[i] = review['text']
                i += 1
        if 'uiyelp' in loaded_json:
            reviews = loaded_json['uiyelp']['reviews']
            for review in reviews:
                text_dictionary[i] = review['text']
                i += 1
        line = f.readline()
#print(text_dictionary[1])

# Data Preprocessing II

Expand contractions

In [ ]:
#code source is http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "youyou will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text.lower())
    return text

In [ ]:
for i in range(0,len(text_dictionary)):
    text_dictionary[i]=expandContractions(text_dictionary[i])

Remove spcial or accented characters (and maybe numbers)

In [ ]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [ ]:
for i in range(0,len(text_dictionary)):
    text_dictionary[i]=remove_special_characters(text_dictionary[i], 
                          remove_digits=True)

Stopwords

In [ ]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [ ]:
for i in range(0,len(text_dictionary)):
    text_dictionary[i]=remove_stopwords(text_dictionary[i])

Stemming / Lemming

In [ ]:
ps = nltk.porter.PorterStemmer()
def simple_stemmer(text):
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [ ]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [ ]:
for i in range(0,len(text_dictionary)):
    text_dictionary[i]=simple_stemmer(text_dictionary[i])

In [ ]:
text_dictionary

Tokenization

In [ ]:
words_corpus = []
for i in range(0,len(text_dictionary)):
        words_corpus.append([word for word in text_dictionary[i].lower().split() if len(word)>3])

In [ ]:
dictionary = corpora.Dictionary(words_corpus)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
print(dictionary)

Bag of Words


In [ ]:
corpus_bow = [dictionary.doc2bow(text) for text in words_corpus]

Term Frequency - Inverse Document Frequency

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

# Topic Modeling

Topic modeling using Bag of Words

In [ ]:
lda_model = models.LdaMulticore(corpus_bow, num_topics=3, id2word=dictionary, passes=1, workers=5)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
for index, score in sorted(lda_model[corpus_bow[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
lda_model.save("lda_model_all")

Topic modeling using TF-IDF

In [ ]:
lda_model_tfidf = models.LdaMulticore(corpus_tfidf, num_topics=3, id2word=dictionary, passes=1, workers=5)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
for index, score in sorted(lda_model_tfidf[corpus_bow[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

In [ ]:
lda_model_tfidf.save("lda_model_tfidf_all")